# Reaction Classification Project - Dataset Discovery

**Goal:** Find and explore ORD datasets for Suzuki-Miyaura coupling and C-N coupling reactions to build a reaction classifier.

---

In [1]:
# Core imports
from ord_schema import message_helpers
from ord_schema.proto import dataset_pb2, reaction_pb2
from glob import glob
import gzip
import os
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

## 1. Scan ORD Repository for Datasets

First, let's find all available datasets and check their names/descriptions for our target reaction types.

In [2]:
# Set path to your ord-data repository
# UPDATE THIS PATH if your repo is in a different location
ORD_DATA_PATH = "ord-data/data"

# Find all dataset files
dataset_files = glob(os.path.join(ORD_DATA_PATH, "**/*.pb.gz"), recursive=True)
print(f"Found {len(dataset_files)} dataset files")

Found 546 dataset files


In [3]:
# Keywords to search for in dataset names/descriptions
SUZUKI_KEYWORDS = ["suzuki", "miyaura", "suzuki-miyaura", "boronic", "boronate"]
CN_KEYWORDS = ["c-n coupling", "c-n bond", "buchwald", "hartwig", "amination", "ullmann"]

def check_keywords(text, keywords):
    """Check if any keyword is in the text (case-insensitive)."""
    if text is None:
        return False
    text_lower = text.lower()
    return any(kw in text_lower for kw in keywords)

def get_dataset_info(filepath):
    """Load dataset and extract basic info."""
    try:
        dataset = message_helpers.load_message(filepath, dataset_pb2.Dataset)
        return {
            "filepath": filepath,
            "dataset_id": dataset.dataset_id,
            "name": dataset.name,
            "description": dataset.description,
            "num_reactions": len(dataset.reactions)
        }
    except Exception as e:
        return None

In [4]:
# Scan all datasets and categorize
suzuki_datasets = []
cn_datasets = []
other_datasets = []

print("Scanning datasets (this may take a few minutes)...")

for filepath in tqdm(dataset_files):
    info = get_dataset_info(filepath)
    if info is None:
        continue
    
    # Check name and description for keywords
    text_to_check = f"{info['name']} {info['description']}"
    
    if check_keywords(text_to_check, SUZUKI_KEYWORDS):
        suzuki_datasets.append(info)
    elif check_keywords(text_to_check, CN_KEYWORDS):
        cn_datasets.append(info)
    else:
        other_datasets.append(info)

print(f"\nFound:")
print(f"  - Suzuki-Miyaura datasets: {len(suzuki_datasets)}")
print(f"  - C-N coupling datasets: {len(cn_datasets)}")
print(f"  - Other datasets: {len(other_datasets)}")

Scanning datasets (this may take a few minutes)...


  0%|          | 0/546 [00:00<?, ?it/s]

100%|██████████| 546/546 [00:28<00:00, 18.86it/s]


Found:
  - Suzuki-Miyaura datasets: 9
  - C-N coupling datasets: 4
  - Other datasets: 533


## 2. Explore Found Datasets

In [5]:
# Display Suzuki-Miyaura datasets with folder paths
print("=" * 80)
print("SUZUKI-MIYAURA DATASETS")
print("=" * 80)

suzuki_df = pd.DataFrame(suzuki_datasets)
if len(suzuki_df) > 0:
    suzuki_df = suzuki_df.sort_values("num_reactions", ascending=False)
    for i, row in suzuki_df.iterrows():
        # Extract subfolder from filepath
        filepath = row['filepath']
        subfolder = filepath.split('/')[-2]  # Gets "3b", "68", etc.
        filename = filepath.split('/')[-1]   # Gets the .pb.gz filename
        
        print(f"\n📁 {row['name'][:80]}..." if len(str(row['name'])) > 80 else f"\n📁 {row['name']}")
        print(f"   Reactions: {row['num_reactions']}")
        print(f"   Path: ord-data/data/{subfolder}/{filename}")
        desc = str(row['description'])[:150] + "..." if len(str(row['description'])) > 150 else row['description']
        print(f"   Description: {desc}")
else:
    print("No Suzuki-Miyaura datasets found with current keywords.")

SUZUKI-MIYAURA DATASETS

📁 Chan-Lam coupling of primary sulfonamides with boronic acids
   Reactions: 9632
   Path: ord-data/data/ord-data/data\5c\ord_dataset-5c9a10329a8a48968d18879a48bb8ab2.pb.gz
   Description: Screen of Chan-Lam couplings of primary sulfonamides with boronic acids, from 10.26434/chemrxiv-2024-22jrq, by the Abigail Doyle group in collaboratio...

📁 Rapid material-sparing screening of 5760 Suzuki-Miyaura coupling reactions
   Reactions: 5760
   Path: ord-data/data/ord-data/data\68\ord_dataset-68cb8b4b2b384e3d85b5b1efae58b203.pb.gz
   Description: 5760 nanoscale flow reactions from https://doi.org/10.1126/science.aap9112 screening a series of electrophiles and nucleophiles against 12 ligands (on...

📁 Cernak miniaturized Suzuki Cross-Coupling 12x10 Informer Library
   Reactions: 1440
   Path: ord-data/data/ord-data/data\3b\ord_dataset-3b8a2ef300e145468579027f206a3ac8.pb.gz
   Description: Miniaturized Suzuki cross-coupling of 12 aryl halide cores, with 10 boronic acid

In [6]:
# Display C-N coupling datasets
print("=" * 80)
print("C-N COUPLING DATASETS")
print("=" * 80)

cn_df = pd.DataFrame(cn_datasets)
if len(cn_df) > 0:
    cn_df = cn_df.sort_values("num_reactions", ascending=False)
    for i, row in cn_df.iterrows():
        print(f"\n📁 {row['name'][:80]}..." if len(str(row['name'])) > 80 else f"\n📁 {row['name']}")
        print(f"   Reactions: {row['num_reactions']}")
        print(f"   ID: {row['dataset_id']}")
        desc = str(row['description'])[:150] + "..." if len(str(row['description'])) > 150 else row['description']
        print(f"   Description: {desc}")
else:
    print("No C-N coupling datasets found with current keywords.")

C-N COUPLING DATASETS

📁 Cernak miniaturized reductive amination
   Reactions: 1152
   ID: ord_dataset-2be11f57f3304e678ea8469baf6dd1bc
   Description: Miniaturized reductive aminations of staurosporine with aldehydes and ketones. From Time Cernak and Merck publication 10.1038/s44160-023-00351-1. Cont...

📁 750 AstraZeneca ELN dataset
   Reactions: 750
   ID: ord_dataset-00005539a1e04c809a9a78647bea649c
   Description: This dataset includes 750 Buchwald-Hartwig reactions generated from AstraZeneca ELN which is used in yield prediction work: 10.26434/chemrxiv.14589498...

📁 Pd-catalyzed Buchwald-Hartwig C-N cross-coupling
   Reactions: 288
   ID: ord_dataset-cbcc4048add7468e850b6ec42549c70d
   Description: A palladium-catalyzed Buchwald-Hartwig cross coupling reaction with different precatalysts with different ligands, and bases.

📁 Copper-Catalyzed Enantioselective Hydroamination of Alkenes
   Reactions: 3
   ID: ord_dataset-5540e162c09f4c04905ddc8ba9c931c6
   Description: Reaction dat

In [7]:
# Summary statistics
print("\n" + "=" * 80)
print("SUMMARY")
print("=" * 80)

total_suzuki = sum(d['num_reactions'] for d in suzuki_datasets)
total_cn = sum(d['num_reactions'] for d in cn_datasets)

print(f"\nSuzuki-Miyaura:")
print(f"  - {len(suzuki_datasets)} datasets")
print(f"  - {total_suzuki} total reactions")

print(f"\nC-N Coupling:")
print(f"  - {len(cn_datasets)} datasets")
print(f"  - {total_cn} total reactions")

print(f"\nCombined: {total_suzuki + total_cn} reactions for classification")


SUMMARY

Suzuki-Miyaura:
  - 9 datasets
  - 18043 total reactions

C-N Coupling:
  - 4 datasets
  - 2193 total reactions

Combined: 20236 reactions for classification


## 3. Select Datasets for Training

Based on the above exploration, select which datasets to use. Ideally pick datasets that:
- Have enough reactions (>100 each)
- Have yield data available
- Are reasonably balanced between classes

In [8]:
# After reviewing the output above, manually select the datasets you want to use
# Fill in the filepaths from the discovered datasets

SELECTED_SUZUKI = [
    # Add filepaths here, e.g.:
    # "ord-data/data/xx/ord_dataset-xxxxx.pb.gz",
]

SELECTED_CN = [
    # Add filepaths here, e.g.:
    # "ord-data/data/xx/ord_dataset-xxxxx.pb.gz",
]

print(f"Selected {len(SELECTED_SUZUKI)} Suzuki datasets")
print(f"Selected {len(SELECTED_CN)} C-N coupling datasets")

Selected 0 Suzuki datasets
Selected 0 C-N coupling datasets


## 4. Deep Dive: Inspect a Dataset

Before committing, let's look at the structure of a specific dataset to understand what features are available.

In [9]:
def inspect_dataset(filepath, n_samples=3):
    """Inspect the structure and content of a dataset."""
    dataset = message_helpers.load_message(filepath, dataset_pb2.Dataset)
    
    print(f"Dataset: {dataset.name}")
    print(f"Total reactions: {len(dataset.reactions)}")
    print("\n" + "-" * 40)
    
    # Check what fields are populated across reactions
    has_yield = 0
    has_temperature = 0
    has_time = 0
    input_roles = defaultdict(int)
    
    for rxn in dataset.reactions:
        # Check for yield
        for outcome in rxn.outcomes:
            for product in outcome.products:
                for m in product.measurements:
                    if m.type == reaction_pb2.ProductMeasurement.YIELD:
                        has_yield += 1
                        break
        
        # Check conditions
        if rxn.conditions.HasField("temperature"):
            has_temperature += 1
        
        # Check for reaction time in outcomes
        for outcome in rxn.outcomes:
            if outcome.HasField("reaction_time"):
                has_time += 1
                break
        
        # Count input roles
        for key, inp in rxn.inputs.items():
            for comp in inp.components:
                role = reaction_pb2.ReactionRole.ReactionRoleType.Name(comp.reaction_role)
                input_roles[role] += 1
    
    n = len(dataset.reactions)
    print(f"\nData availability:")
    print(f"  - Yield data: {has_yield}/{n} ({100*has_yield/n:.1f}%)")
    print(f"  - Temperature: {has_temperature}/{n} ({100*has_temperature/n:.1f}%)")
    print(f"  - Reaction time: {has_time}/{n} ({100*has_time/n:.1f}%)")
    
    print(f"\nInput roles found:")
    for role, count in sorted(input_roles.items(), key=lambda x: -x[1]):
        print(f"  - {role}: {count}")
    
    # Show sample reactions
    print(f"\n" + "=" * 40)
    print(f"Sample reactions:")
    print("=" * 40)
    
    for i, rxn in enumerate(dataset.reactions[:n_samples]):
        print(f"\n--- Reaction {i+1} ---")
        for key, inp in rxn.inputs.items():
            print(f"  Input '{key}':")
            for comp in inp.components:
                role = reaction_pb2.ReactionRole.ReactionRoleType.Name(comp.reaction_role)
                # Get identifier
                ident_str = "[no identifier]"
                for ident in comp.identifiers:
                    ident_type = reaction_pb2.CompoundIdentifier.CompoundIdentifierType.Name(ident.type)
                    if ident_type in ["SMILES", "NAME"]:
                        ident_str = f"{ident_type}: {ident.value[:50]}{'...' if len(ident.value) > 50 else ''}"
                        break
                print(f"    - {role}: {ident_str}")
    
    return dataset

In [10]:
# Inspect one of the found Suzuki datasets (if any)
if len(suzuki_datasets) > 0:
    # Pick the largest one
    largest_suzuki = max(suzuki_datasets, key=lambda x: x['num_reactions'])
    print("Inspecting largest Suzuki-Miyaura dataset:\n")
    _ = inspect_dataset(largest_suzuki['filepath'])

Inspecting largest Suzuki-Miyaura dataset:

Dataset: Chan-Lam coupling of primary sulfonamides with boronic acids
Total reactions: 9632

----------------------------------------

Data availability:
  - Yield data: 19264/9632 (200.0%)
  - Temperature: 9632/9632 (100.0%)
  - Reaction time: 9632/9632 (100.0%)

Input roles found:
  - REACTANT: 19264
  - SOLVENT: 19264
  - CATALYST: 9632
  - REAGENT: 8112

Sample reactions:

--- Reaction 1 ---
  Input 'Boronic Acid':
    - REACTANT: SMILES: OB(O)C1=CC=C(C(F)(F)F)C=C1
    - SOLVENT: SMILES: ClCCCl
  Input 'Sulfonamide':
    - REACTANT: SMILES: BrC1=CC(S(=O)(N)=O)=CC(C(OC)=O)=C1
    - SOLVENT: SMILES: ClCCCl
  Input 'Base_Solid':
    - REAGENT: SMILES: [F-].[Cs+]
  Input 'Catalyst':
    - CATALYST: SMILES: CC(=O)[O-].CC(=O)[O-].[Cu+2]

--- Reaction 2 ---
  Input 'Boronic Acid':
    - REACTANT: SMILES: OB(O)C1=CC=C(OC)C=C1
    - SOLVENT: SMILES: ClCCCl
  Input 'Sulfonamide':
    - REACTANT: SMILES: BrC1=CC(S(=O)(N)=O)=CC(C(OC)=O)=C1
    - SOLV

In [11]:
# Inspect one of the found C-N coupling datasets (if any)
if len(cn_datasets) > 0:
    # Pick the largest one
    largest_cn = max(cn_datasets, key=lambda x: x['num_reactions'])
    print("Inspecting largest C-N coupling dataset:\n")
    _ = inspect_dataset(largest_cn['filepath'])

Inspecting largest C-N coupling dataset:

Dataset: Cernak miniaturized reductive amination
Total reactions: 1152

----------------------------------------

Data availability:
  - Yield data: 0/1152 (0.0%)
  - Temperature: 1152/1152 (100.0%)
  - Reaction time: 1152/1152 (100.0%)

Input roles found:
  - SOLVENT: 8064
  - REAGENT: 3456
  - REACTANT: 2304

Sample reactions:

--- Reaction 1 ---
  Input 'acetic acid':
    - REAGENT: NAME: acetic acid
    - SOLVENT: NAME: DMA
  Input 'STAB solution':
    - REAGENT: NAME: Sodium triacetoxyborohydride
    - SOLVENT: NAME: DMA
  Input 'aldehyde/ketone solution':
    - REACTANT: SMILES: N#Cc1ccc(C=O)cc1Br
    - SOLVENT: NAME: DMA
  Input 'Ti(OiPr)4 solution':
    - REAGENT: NAME: Ti(OiPr)4
    - SOLVENT: NAME: DMA
  Input 'dilution solvent 2':
    - SOLVENT: NAME: DMA
  Input 'dilution solvent 1':
    - SOLVENT: NAME: DMA
  Input 'core solution':
    - REACTANT: NAME: staurosporine
    - SOLVENT: NAME: DMA

--- Reaction 2 ---
  Input 'acetic acid

In [12]:
# Check AstraZeneca dataset
inspect_dataset("ord-data/data/00/ord_dataset-00005539a1e04c809a9a78647bea649c.pb.gz")

# Check Pd-catalyzed Buchwald-Hartwig
inspect_dataset("ord-data/data/cb/ord_dataset-cbcc4048add7468e850b6ec42549c70d.pb.gz")

Dataset: 750 AstraZeneca ELN dataset
Total reactions: 750

----------------------------------------

Data availability:
  - Yield data: 750/750 (100.0%)
  - Temperature: 750/750 (100.0%)
  - Reaction time: 0/750 (0.0%)

Input roles found:
  - REACTANT: 1500
  - CATALYST: 1498
  - REAGENT: 750
  - SOLVENT: 741

Sample reactions:

--- Reaction 1 ---
  Input 'Base':
    - REAGENT: SMILES: C(=O)([O-])[O-].[Cs+].[Cs+]
  Input 'metal and ligand':
    - CATALYST: SMILES: C1=CC=C(C=C1)P(C2=CC=CC=C2)C3=C(C4=CC=CC=C4C=C3)C5...
    - CATALYST: SMILES: C1=CC=C(C=C1)/C=C/C(=O)/C=C/C2=CC=CC=C2.C1=CC=C(C=...
  Input 'aryl halide':
    - REACTANT: SMILES: CCOC1=C(C=C2C(=C1)N=CC(=C2NC3=C(C=C(C=C3)F)F)C(=O)...
  Input 'amine':
    - REACTANT: SMILES: CC(C)N1CCNCC1

--- Reaction 2 ---
  Input 'Base':
    - REAGENT: SMILES: C(=O)([O-])[O-].[Cs+].[Cs+]
  Input 'Solvent':
    - SOLVENT: SMILES: COC1=CC=CC=C1
  Input 'metal and ligand':
    - CATALYST: SMILES: CC1(C2=C(C(=CC=C2)P(C3=CC=CC=C3)C4=CC=CC=C4)OC5=

name: "Pd-catalyzed Buchwald-Hartwig C-N cross-coupling"
description: "A palladium-catalyzed Buchwald-Hartwig cross coupling reaction with different precatalysts with different ligands, and bases."
reactions {
  identifiers {
    type: REACTION_TYPE
    value: "C-N cross coupling"
  }
  inputs {
    key: "Pd precatalyst (stock solution in DMSO)"
    value {
      components {
        identifiers {
          type: SMILES
          value: "CS(=O)(=O)O.C1=CC=C(C=C1)[PH+](C2=CC=CC=C2)C3=C(C4=CC=CC=C4C=C3)C5=C(C=CC6=CC=CC=C65)P(C7=CC=CC=C7)C8=CC=CC=C8.C1=CC=C([C-]=C1)C2=CC=CC=C2N.[Pd]"
        }
        amount {
          moles {
            value: 10
            units: NANOMOLE
          }
        }
        reaction_role: CATALYST
      }
      components {
        identifiers {
          type: NAME
          value: "DMSO"
        }
        identifiers {
          type: SMILES
          value: "CS(=O)C"
        }
        amount {
          volume {
            value: 0.25
            units

In [13]:
import os
import glob
from ord_schema import message_helpers
from ord_schema.proto import dataset_pb2

# Find all dataset files
all_datasets = []
dataset_files = glob.glob("ord-data/data/**/*.pb.gz", recursive=True)

print(f"Scanning {len(dataset_files)} dataset files...")

for filepath in tqdm(dataset_files):
    try:
        dataset = message_helpers.load_message(filepath, dataset_pb2.Dataset)
        all_datasets.append({
            'name': dataset.name,
            'num_reactions': len(dataset.reactions),
            'description': dataset.description[:100] if dataset.description else "",
            'filepath': filepath
        })
    except Exception as e:
        continue

# Sort by number of reactions
df_all = pd.DataFrame(all_datasets)
df_all = df_all.sort_values('num_reactions', ascending=False)

# Show top 10
print("\n" + "="*70)
print("TOP 10 LARGEST DATASETS IN ORD")
print("="*70)

for i, row in df_all.head(10).iterrows():
    subfolder = row['filepath'].split('/')[-2]
    filename = row['filepath'].split('/')[-1]
    print(f"\n{row['num_reactions']:,} reactions - {row['name'][:60]}")
    print(f"   Path: ord-data/data/{subfolder}/{filename}")
    print(f"   Desc: {row['description']}...")

Scanning 546 dataset files...


100%|██████████| 546/546 [00:43<00:00, 12.53it/s]


TOP 10 LARGEST DATASETS IN ORD

47,015 reactions - AIChemEco amide coupling conditions 47k dataset
   Path: ord-data/data/ord-data/data\47\ord_dataset-47eaacc46c3a4487bbdf99adb1a15e41.pb.gz
   Desc: A high-throughput study of amide coupling conditions by AIChemEco from DOI: 10.1039/d5sc03364k. Data...

40,904 reactions - Training data from https://doi.org/10.1039/C8SC04228D (4/10)
   Path: ord-data/data/ord-data/data\e5\ord_dataset-e52a0f66fdfa4d2ebbb136996350b697.pb.gz
   Desc: 409035 reaction SMILES downloaded from https://github.com/connorcoley/rexgen_direct (4/10)...

40,904 reactions - Training data from https://doi.org/10.1039/C8SC04228D (2/10)
   Path: ord-data/data/ord-data/data\0c\ord_dataset-0c1e1da868cd46c59efce6d5d4bc3d63.pb.gz
   Desc: 409035 reaction SMILES downloaded from https://github.com/connorcoley/rexgen_direct (2/10)...

40,904 reactions - Training data from https://doi.org/10.1039/C8SC04228D (1/10)
   Path: ord-data/data/ord-data/data\9f\ord_dataset-9fc3edef9c184

In [14]:
# Search for different reaction types in ORD
search_terms = {
    "suzuki": ["suzuki", "chan-lam"],
    "buchwald": ["buchwald", "hartwig", "c-n coupling", "c-n cross"],
    "amination": ["amination", "reductive amination"],
    "heck": ["heck"],
    "negishi": ["negishi"],
    "sonogashira": ["sonogashira"],
}

import glob

dataset_files = glob.glob("ord-data/data/**/*.pb.gz", recursive=True)

print(f"Scanning {len(dataset_files)} datasets for reaction types...\n")

for reaction_type, keywords in search_terms.items():
    matches = []
    for filepath in dataset_files:
        try:
            dataset = message_helpers.load_message(filepath, dataset_pb2.Dataset)
            text = (dataset.name + " " + dataset.description).lower()
            if any(kw in text for kw in keywords):
                matches.append({
                    'name': dataset.name,
                    'count': len(dataset.reactions),
                    'path': filepath
                })
        except:
            continue
    
    print(f"\n{'='*50}")
    print(f"{reaction_type.upper()} ({len(matches)} datasets)")
    print(f"{'='*50}")
    for m in sorted(matches, key=lambda x: x['count'], reverse=True):
        subfolder = m['path'].split('/')[-2]
        filename = m['path'].split('/')[-1]
        print(f"  {m['count']:>6} - {m['name'][:50]}")
        print(f"         ord-data/data/{subfolder}/{filename}")

Scanning 546 datasets for reaction types...


SUZUKI (9 datasets)
    9632 - Chan-Lam coupling of primary sulfonamides with bor
         ord-data/data/ord-data/data\5c\ord_dataset-5c9a10329a8a48968d18879a48bb8ab2.pb.gz
    5760 - Rapid material-sparing screening of 5760 Suzuki-Mi
         ord-data/data/ord-data/data\68\ord_dataset-68cb8b4b2b384e3d85b5b1efae58b203.pb.gz
    1440 - Cernak miniaturized Suzuki Cross-Coupling 12x10 In
         ord-data/data/ord-data/data\3b\ord_dataset-3b8a2ef300e145468579027f206a3ac8.pb.gz
     450 - Ni-catalyzed Suzuki Miyaura cross-coupling
         ord-data/data/ord-data/data\3b\ord_dataset-3b5db90e337942ea886b8f5bc5e3aa72.pb.gz
     376 - HTE Suzuki coupling dataset
         ord-data/data/ord-data/data\ee\ord_dataset-eeba974d3c284aed86d1c1d442260a1e.pb.gz
      97 - Reizman Suzuki Cross-Coupling
         ord-data/data/ord-data/data\2b\ord_dataset-2be30f5d8dcd471aa6ad410bdee05902.pb.gz
      96 - Reizman Suzuki Cross-Coupling
         ord-data/data/ord-

## 5. Next Steps

After running this notebook, you will know:
1. How many datasets exist for each reaction type
2. How many reactions are available
3. What features are consistently available (yield, temperature, roles, etc.)

The next notebook will:
1. Load the selected datasets
2. Extract features (SMILES → fingerprints or one-hot encoding)
3. Build and train the classification model

---

**Note:** If the keyword search doesn't find enough datasets, we may need to:
- Expand the keywords
- Look at the reaction SMILES patterns directly
- Check the `reaction_type` field in the reaction messages